# Text Generation

In this notebook, we demonstrate how the text prompts are created based on the information in the tabular patient record. These prompts were fed into GPT-4o to generate our free text descriptions of the patient encounters.  

In [1]:
from utils.prompt_generation import PromptGeneration, prompt_GPT_no_symptom_no_bg, split_unrelated_notes

We load in the SynSUM dataset, which already includes the synthetic text notes (columns "text" and "advanced_text"). We isolate the tabular portion of the dataset to demonstrate how we generated the prompts that were used to obtain the synthetic text notes. 

In [2]:
import pickle
with open("data/df_synsum.p", "rb") as file: 
    df = pickle.load(file)

In [3]:
df_tabular = df[["policy", "self_empl", "asthma", "smoking", "COPD", "winter", "hay_fever", "pneu", "inf", "dysp", "cough", "pain", "fever", "nasal", "antibiotics", "days_at_home"]]
df_tabular.head()

,policy,self_empl,asthma,smoking,COPD,winter,hay_fever,pneu,inf,dysp,cough,pain,fever,nasal,antibiotics,days_at_home
0,yes,no,no,no,no,no,no,no,no,no,no,no,high,no,yes,2
1,yes,no,no,no,no,no,no,no,no,no,no,no,none,no,no,2
2,yes,no,no,no,no,no,no,no,no,no,no,no,low,no,no,3
3,no,yes,no,no,no,no,yes,no,no,no,no,no,low,yes,no,2
4,yes,no,no,no,no,yes,no,no,no,no,no,no,none,no,no,1


We first generate the prompts for the portion of the patients where at least one symptom is positive. There are 6371 patients like this (out of 10000 total). 

In [4]:
# make prompt generation object
prompt_gen = PromptGeneration(df_tabular, 2024)

# df containing prompts for the patients where at least one symptom is positive
df_prompt_pos = prompt_gen.generate_prompts_positive() 

Additional columns were added to the dataframe, to help with the construction of the prompt: 
- {symptom}_mention: True/False (whether symptom should be mentioned in note or not)
- {symptom}_descr: descriptor added to prompt, describing the symptom conditional on its cause
- symptoms_string: list of symptoms to be used as first part of the prompt
- no_mention_symptoms_string: list of symptoms to be used as second part of the prompt
- background_string: list of underlying background conditions. 

All these elements are used to build up the prompt. For example, the prompt for the first patient (ID: 0) is printed below. 

In [5]:
len(df_prompt_pos)

6371

In [6]:
df_prompt_pos.head()

,asthma,smoking,COPD,hay_fever,pneu,inf,dysp,cough,pain,fever,...,pain_mention,fever_mention,nasal_mention,dysp_descr,cough_descr,pain_descr,symptoms_string,no_mention_symptoms_string,background_string,prompt
0,no,no,no,no,no,no,no,no,no,high,...,True,True,False,,,,- respiratory pain: no\n- fever: high\n- cough...,- nasal symptoms\n,,Create a short clinical note related to the fo...
2,no,no,no,no,no,no,no,no,no,low,...,False,False,False,,,,- cough: no\n- dyspnea: no\n,- respiratory pain\n- nasal symptoms\n- fever\n,,Create a short clinical note related to the fo...
3,no,no,no,yes,no,no,no,no,no,low,...,True,True,True,,,,- nasal symptoms: yes\n- fever: low\n- respira...,,- hay fever\n,Create a short clinical note related to the fo...
5,no,no,no,no,no,yes,no,yes,yes,high,...,True,True,True,,light,light,- nasal symptoms: yes\n- respiratory pain: yes...,,,Create a short clinical note related to the fo...
6,yes,no,no,no,no,no,yes,yes,yes,low,...,True,True,False,air hunger,dry,burning pain in windpipe,"- cough: yes, dry\n- fever: low\n- respiratory...",- nasal symptoms\n,- asthma\n,Create a short clinical note related to the fo...


In [7]:
print(df_prompt_pos.loc[0]["prompt"])

Create a short clinical note related to the following patient encounter. 

The following information is known about the patient's symptoms:
- respiratory pain: no
- fever: high
- cough: no
- dyspnea: no

Don't mention anything about the following symptoms:
- nasal symptoms

The note has the following structure: 
**History**
<history>
**Physical Examination**
<physical examination results>

Do not include any suspicions of possible diagnoses in the clinical note (no "assessment" field). You can imagine additional context or details described by the patient, but no additional symptoms. Do not mention patient gender or age. Your notes can be relatively long (around 5 lines or more in history).

Do not add a title. Do not add a final comment after generating the note. 


We can then pass this prompt to GPT-4o using the ChatCompletions API. We used the "prompt_GPT" function in prompt_generation.py to generate our texts. We don't demonstrate its use here, since you need to log in to OpenAI with your own API key to make it work. 

The remaining 3269 patients had symptoms that were all negative (all "no"). Of these patients, 239 had at least one underlying respiratory condition. In this case, we use a similar prompt compared to the patients which had at least one positive symptom. Again, we can use the "prompt_GPT" function to generate a text note. 

In [8]:
# df containing prompts for the patients where all symptoms are negative, but patient has at least one 
# underlying respiratory condition
df_prompt_neg_bg = prompt_gen.generate_prompts_negative_bg()

In [9]:
len(df_prompt_neg_bg)

239

In [12]:
print(df_prompt_neg_bg.iloc[0]["prompt"])

Create a short clinical note related to the following patient encounter.
            
The patient does not experience any of the following symptoms:
- fever
- dyspnea
- chest pain / pain attributed to airways
- sneezing / blocked nose
- cough
            
The patient currently has the following underlying health conditions, which may or may not be mentioned in the note if relevant:
- smoking

The note has the following structure: 
**History**
<history>
**Physical Examination**
<physical examination results>

Do not include any suspicions of possible diagnoses in the clinical note (no "assessment" field). You can imagine context or details described by the patient. Do not mention patient gender or age. Your notes can be relatively long (around 5 lines or more in history).

Do not add a title. Do not add a final comment after generating the note. 


Finally, there are 3390 patients left, which do not have any positive symptoms, and no underlying respiratory conditions either. In this case, we want to give the LLM a little more freedom to invent another reason of visit, unrelated to respiratory diseases. We use a different prompt to generate these notes. 

In [22]:
# select all patients without any positive symptoms, and without any underlying respiratory condition
df_neg_symptoms = df_tabular.loc[(df_tabular["dysp"]=="no")&(df_tabular["cough"]=="no")&(df_tabular["pain"]=="no")&(df_tabular["fever"]=="none")&(df_tabular["nasal"]=="no")].copy()
df_neg_symptoms_no_bg = df_neg_symptoms.loc[(df_neg_symptoms["asthma"]=="no")&(df_neg_symptoms["smoking"]=="no")&(df_neg_symptoms["COPD"]=="no")&(df_neg_symptoms["hay_fever"]=="no")].copy()

In [23]:
len(df_neg_symptoms_no_bg)

3390

We use the following prompt to generate these unrelated text notes. Note that we ask to generate 3 at the same time, to encourage more creativity and variation. We are able to do this now, since the prompt is NOT customized to each patient (these patients all have the same set of symptoms and underlying conditions, so there is nothing to customize). 

In [17]:
base_prompt = """Create 3 short clinical notes related to the following patient encounter.
 
The patient does not experience any of the following symptoms:
- fever
- dyspnea
- chest pain / pain attributed to airways
- sneezing / blocked nose
- cough

The patient does not have any of the following health conditions, so don't mention these: 
- asthma
- COPD
- smoking
- hay fever

The note has the following structure: 
**History**
<history>
**Physical Examination**
<physical examination results>

Do not include any suspicions of possible diagnoses in the clinical note (no "assessment" field). You can imagine context or details described by the patient. Do not mention patient gender or age. Your notes can be relatively long (around 5 lines or more in history).

Separate the individual notes with "---". Do not add a title. Do not add a final comment after generating the note. """

print(base_prompt)

Create 3 short clinical notes related to the following patient encounter.
 
The patient does not experience any of the following symptoms:
- fever
- dyspnea
- chest pain / pain attributed to airways
- sneezing / blocked nose
- cough

The patient does not have any of the following health conditions, so don't mention these: 
- asthma
- COPD
- smoking
- hay fever

The note has the following structure: 
**History**
<history>
**Physical Examination**
<physical examination results>

Do not include any suspicions of possible diagnoses in the clinical note (no "assessment" field). You can imagine context or details described by the patient. Do not mention patient gender or age. Your notes can be relatively long (around 5 lines or more in history).

Separate the individual notes with "---". Do not add a title. Do not add a final comment after generating the note. 


We now use the "prompt_GPT_no_symptom_no_bg" to prompt the LLM for these special cases. The LLM will return 3 notes at once, so we need to split them up. For this, the function "split_unrelated_notes" is used. It throws up an error message if the requested format for the 3 notes is not followed, in which case the notes cannot be separated and the LLM's response can be discarded. 

The code below demonstrates how we would build up the full list of all so-called unrelated notes. This process was repeated until 3390 unrelated notes were obtained. These were then randomly added to the dataframe. To be able to run the code below, you must fill in your own OpenAI access token. 

In [ ]:
# prompt three notes at once, because prompt is the same for every patient in df_neg_symptoms_no_bg
# here, we do this 5 times, but you can do this as many times as needed to fill up the dataframe
all_responses = []
for i in range(5):
    response = prompt_GPT_no_symptom_no_bg()
    all_responses.append(response)
unrelated_notes = []
for response in all_responses:
    unrelated_notes += split_unrelated_notes(response)
# once you have gathered enough unrelated_notes, you can add them to the dataframe randomly 
# for any note in df_neg_symptoms_no_bg
df_neg_symptoms_no_bg.loc[1, "text_note"] = unrelated_notes[0]

After all text notes were generated and filled in in the dataset, the three dataframes are merged again (in order). 

In [24]:
import pandas as pd
# at the end, merge the dataframes again
df_total = pd.concat([df_prompt_pos, df_prompt_neg_bg, df_neg_symptoms_no_bg], sort=False).sort_index()

Once we had a dataframe filled with normal text notes, we generated the compact text notes. For this, we simply provided the original prompt together with the model's response, after which we asked the LLM to write the same note in more compact style. This is encoded in the function "prompt_GPT_advanced_note". Once again, the code can be ran by specifying your own OpenAI access token. 

The full prompt for every patient, together with the helper columns used to build the prompt, can be found in the pickled dataframe "data/df_prompts.p". 

In [41]:
with open("data/df_prompts.p", "rb") as file: 
    df_prompts = pickle.load(file)

In [42]:
df_prompts.iloc[556]

asthma                                                                      yes
smoking                                                                      no
COPD                                                                         no
hay_fever                                                                    no
pneu                                                                         no
inf                                                                         yes
dysp                                                                        yes
cough                                                                       yes
pain                                                                        yes
fever                                                                      none
nasal                                                                        no
dysp_mention                                                               True
cough_mention                           

In [44]:
print(df_prompts.iloc[556]["text_note"])

**History**
The patient presents with a persistent cough that occurs both day and night. Additionally, they report episodes of dyspnea associated with these attacks. The patient describes a burning pain localized to the trachea. The onset of symptoms seems to exacerbate their known asthma, which has been previously managed. The cough is non-productive, and lifestyle factors or recent travel were denied.

**Physical Examination**
Vital signs are within normal limits. No noticeable nasal congestion or rhinorrhea. Respiratory examination reveals intermittent wheezing and prolonged expiratory phase. There is no use of accessory muscles observed. The trachea is midline with no tenderness upon palpation. Thoracic auscultation confirms decreased air entry on forced expiration. No cyanosis or clubbing detected.


In [45]:
print(df_prompts.iloc[556]["advanced_note"])

**History**
Pt presents with persistent cough, day & night. Reports dyspnea linked to attacks and burning pain in trachea. Symptoms exacerbate pt’s known asthma. Non-productive cough. Denies recent travel or lifestyle factors.

**Physical Examination**
VS WNL. No nasal congestion/rhinorrhea. Intermittent wheezing, prolonged expiratory phase noted. No accessory muscle use. Trachea midline, non-tender. Decreased air entry on forced expiration. No cyanosis/clubbing.


In the final SynSUM dataset, we only retain the tabular patient record and the two text notes (normal and compact). The dataframe can be found in "data/df_synsum.p". 

This all culminates in the SynSUM dataset, of which we print the first two records below. 

In [46]:
import pickle
with open("data/df_synsum.p", "rb") as file: 
    df = pickle.load(file)

In [47]:
df.iloc[0]

policy                                                         yes
self_empl                                                       no
asthma                                                          no
smoking                                                         no
COPD                                                            no
winter                                                          no
hay_fever                                                       no
pneu                                                            no
inf                                                             no
dysp                                                            no
cough                                                           no
pain                                                            no
fever                                                         high
nasal                                                           no
antibiotics                                                   

In [33]:
print(df.iloc[0]["text"])

**History**
Patient reports a significant increase in body temperature over the last 48 hours, exceeding normal ranges, indicating a high fever. There have been no respiratory symptoms such as pain, dyspnea, or cough. The patient illustrates general malaise and mentions feeling very fatigued due to the fever. No notable changes in daily routine or exposure to environments that might typically contribute to fever are reported. Recent stress levels and potential exposure to infectious agents during travels are also discussed.

**Physical Examination**
Vital signs show elevated temperature (103 °F). Heart rate is slightly tachycardic at 98 bpm, corresponding with the fever. Oxygen saturation is within normal limits at 98%, and lungs are clear to auscultation without any added sounds. Abdominal examination is normal, without tenderness or organomegaly. Skin shows no rashes, warmth, or lesions. Capillary refill time is adequate. Neurological assessment is non-focal. Overall, there are no ev

In [48]:
print(df.iloc[0]["advanced_text"])

**History**
Pt reports high fever for 48 hrs, denies resp pain, dyspnea, or cough. Describes significant fatigue and malaise. No recent routine changes or known infectious exposures. Discusses recent stress and travel.

**Physical Examination**
VS: Temp 103 °F, HR 98 bpm (tachycardic), O2 sat 98%. Lungs clear, no adventitious sounds. Abd: non-tender, no organomegaly. Skin: no rashes/lesions, normal CRT. Neuro: non-focal. Overall: WNL apart from elevated fever.


In [49]:
df.iloc[1]

policy                                                         yes
self_empl                                                       no
asthma                                                          no
smoking                                                         no
COPD                                                            no
winter                                                          no
hay_fever                                                       no
pneu                                                            no
inf                                                             no
dysp                                                            no
cough                                                           no
pain                                                            no
fever                                                         none
nasal                                                           no
antibiotics                                                   

In [50]:
print(df.iloc[1]["text"])

**History**
The patient reports a gradual onset of fatigue over the past month without any noticeable triggers. Additionally, they describe experiencing intermittent headaches that are relieved with over-the-counter analgesics. Sleep quality has been variable, contributing to a sense of ongoing tiredness upon waking. There is no mention of dietary changes or recent travel. The patient denies experiencing appetite loss or any significant recent weight changes.

**Physical Examination**
Vital signs are stable, with blood pressure at 120/80 mmHg and a pulse of 72 bpm. Examination of the head and neck reveals no abnormalities; the thyroid gland is non-palpable. No lymphadenopathy is noted. Cardiac and respiratory examinations are unremarkable. Abdominal examination shows normal bowel sounds with no tenderness or masses. Neurological assessment reveals no focal deficits.


In [51]:
print(df.iloc[1]["advanced_text"])

**History**
Pt reports gradual onset of fatigue over past month, with intermittent HAs relieved by OTC analgesics. Variable sleep quality, tiredness on waking. No dietary changes, recent travel, appetite loss, or significant recent weight changes.

**Physical Examination**
VS: BP 120/80, HR 72. Head/neck: WNL, no thyroid abnormalities, no LAD. Cardio/resp: Unremarkable. Abd: Normal BS, non-tender, no masses. Neuro: No focal deficits.


## Consistency check

To check that no mistakes were made, we ask GPT-4 to verify once again whether the information in the note corresponds with the information in the prompt. We ask it to output either "Yes." if the note is consistent with the prompt, or "No." if not. If the answer is "No.", we ask GPT4 to provide reasoning for this. For this, we used the prompt below, which we applied to all 10.000 notes. 

In [5]:
import pickle
with open("../data/df_prompts.p", "rb") as file: 
    df = pickle.load(file)

In [6]:
def generate_filtered_prompt(row): 

    # extract patient information
    note = row["text_note"]
    sympt_str = row["symptoms_string"]
    no_mention_sympt_str = row["no_mention_symptoms_string"]
    bg_str = row["background_string"]

    # deal with no observed symptoms instances
    if isinstance(sympt_str, float):
        sympt_str = "- dyspnea: no\n- cough: no\n- respiratory pain: no\n- fever: none\n- nasal symptoms:no\n"
        no_mention_sympt_str = ""
    if isinstance(bg_str, float): 
        bg_str = ""
        for bg in ["asthma", "smoking", "COPD", "hay_fever"]: 
            if row[bg] == "yes": 
                bg_str += f"- {bg}\n"

    # build prompt
    note = note.replace("\n", " ")
    prompt = f"The following information is known about the patient's symptoms. These may or may not be mentioned in the note:\n{sympt_str}\n"
    if len(no_mention_sympt_str) != 0: 
        prompt += f"The following symptoms must not be mentioned:\n{no_mention_sympt_str}\n"
    if len(bg_str) != 0: 
        prompt += f"The patient currently has the following underlying health conditions, which may or may not be mentioned in the note if relevant:\n{bg_str}\n"
    prompt += f"Following the instructions you received, please indicate whether the note below is consistent with the patient information provided above:\n{note}"

    return prompt

In [10]:
prompt = generate_filtered_prompt(df.loc[0])
print(prompt)

The following information is known about the patient's symptoms. These may or may not be mentioned in the note:
- respiratory pain: no
- fever: high
- dyspnea: no
- cough: no

The following symptoms must not be mentioned:
- nasal symptoms

Following the instructions you received, please indicate whether the note below is consistent with the patient information provided above:
**History** Patient reports a significant increase in body temperature over the last 48 hours, exceeding normal ranges, indicating a high fever. There have been no respiratory symptoms such as pain, dyspnea, or cough. The patient illustrates general malaise and mentions feeling very fatigued due to the fever. No notable changes in daily routine or exposure to environments that might typically contribute to fever are reported. Recent stress levels and potential exposure to infectious agents during travels are also discussed.  **Physical Examination** Vital signs show elevated temperature (103 °F). Heart rate is sli

In [ ]:
import openai

SYS_MESSAGE_FILTER = """ 
    I will give you information about a patient's symptoms and some underlying health conditions. I will show you a clinical note that is supposed to describe this patient's encounter with a primary care doctor.
    
    Your job is to decide whether the note is consistent with the information you received on the patient. 
    
    Please answer with "Yes." if it is, and "No." if it is not. If the answer is "No.", explain briefly why not. 
    """
    
def prompt_GPT_filter(patient):

    messages = []

    system_message = {"role": "system", "content": SYS_MESSAGE_FILTER}
    messages.append(system_message)

    messages.append({"role": "user", "content": generate_filtered_prompt(patient)})

    res = openai.chat.completions.create(
        model = "gpt-4o", 
        temperature = 0.2, 
        max_tokens = 1000,
        messages = messages, 
    )

    response = res.choices[0].message.content # response

    return response

We then manually went through the responses and identified 52 notes out of 10.000 that were deemed inconsistent. For these, we asked GPT-4 to generate new notes (using the same prompt as was originally used). We then re-ran the consistency check for these new notes, and found that none of them were inconsistent anymore. We therefore swapped out the old inconsistent notes for their new counterparts, and generated new advanced notes. This left us with the final SynSUM dataset as found in "SynSUM.csv" and all dataframes in the "data" folder. 